In [13]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, StratifiedKFold, cross_val_score
from sklearn.preprocessing import LabelEncoder
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score, precision_score, recall_score, roc_auc_score, confusion_matrix


# Load dataset
data = pd.read_csv('Predict_Academic_Success_Modified.csv')

# Convert categorical columns to numerical using Label Encoding
categorical_columns = ["Marital status", "Application mode", "Course", "Daytime/evening attendance\t", 
                       "Previous qualification", "Nacionality", "Mother's qualification", "Father's qualification"]

label_encoders = {}
for col in categorical_columns:
    le = LabelEncoder()
    data[col] = le.fit_transform(data[col])
    label_encoders[col] = le  # Save encoder in case we need to transform new data

In [2]:
# Convert the target using Label Encoding ('Dropout', 'Enrolled', 'Graduate')
label_encoder = LabelEncoder()
data["Target"] = label_encoder.fit_transform(data["Target"])



In [3]:
# 90% for training/testing, 10% for unseen validation
train_test_data, unseen_data = train_test_split(data, test_size=0.10, random_state=42, stratify=data["Target"])

In [4]:
# Split the 90% into 80% training and 20% testing
train_data, test_data = train_test_split(
    train_test_data, test_size=0.20, random_state=42, stratify=train_test_data["Target"]
)

In [5]:
# Separate features and target variables
X_train = train_data.drop(columns=["Target"])
y_train = train_data["Target"]
X_test = test_data.drop(columns=["Target"])
y_test = test_data["Target"]

In [6]:
# Initialize Naive Bayes model
nb_model = GaussianNB()

In [16]:
# Apply 10-fold cross-validation
cv = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
cross_val_scores = cross_val_score(nb_model, X_train, y_train, cv=cv, scoring='accuracy')
print(f"Cross-validation accuracy scores: \n{cross_val_scores}\n")
print(f"Mean cross-validation accuracy: \n{cross_val_scores.mean()}\n")

# Train the model
nb_model.fit(X_train, y_train)

# Make predictions
y_pred = nb_model.predict(X_test)

# Evaluation metrics
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, average='weighted')
recall = recall_score(y_test, y_pred, average='weighted')
roc_auc = roc_auc_score(y_test, nb_model.predict_proba(X_test), multi_class="ovr")

# Confusion Matrix
conf_matrix = confusion_matrix(y_test, y_pred)

# Create a DataFrame for the evaluation metrics
results_df = pd.DataFrame({
    "Metric": ["Accuracy", "Precision", "Recall", "ROC-AUC"],
    "Score": [accuracy, precision, recall, roc_auc]
})

# Print the results in tabular form
print("\nEvaluation Metrics:")
print(results_df.to_string(index=False))

# Print Confusion Matrix as DataFrame
conf_matrix_df = pd.DataFrame(conf_matrix, 
                              index=label_encoder.classes_, 
                              columns=label_encoder.classes_)

print("\nConfusion Matrix:")
print(conf_matrix_df)

Cross-validation accuracy scores: 
[0.67398119 0.69278997 0.69905956 0.67084639 0.66037736 0.71698113
 0.66352201 0.62578616 0.67610063 0.72327044]

Mean cross-validation accuracy: 
0.6802714851836518


Evaluation Metrics:
   Metric    Score
 Accuracy 0.687578
Precision 0.668846
   Recall 0.687578
  ROC-AUC 0.798770

Confusion Matrix:
          Dropout  Enrolled  Graduate
Dropout       171        33        52
Enrolled       33        39        71
Graduate       32        28       338
